In [2]:
import numpy as np
import pandas as pd

# Flat velocity

In [11]:
# calculates velocity
# no bypass gap
rlc = 1.588/2  # cm
rsc = 1.27/2  # cm
rho = 4.36992  # [kg/m3] 70 bar, 490 C
rho /= 1e6  # [kg/cm3]

m = 0.2  # kg/s
mlc = m/(8.5*rlc**2 + 0.5*rsc**2)*rlc**2
print('mlc: ', mlc, 'kg/s')
msc = m/(8.5*rlc**2 + 0.5*rsc**2)*rsc**2
print('msc: ', msc, 'kg/s')

v = mlc/rho/np.pi/rlc**2
print('v: ', v, 'cm/s')
v = msc/rho/np.pi/rsc**2
print('v: ', v, 'cm/s')

mlc:  0.022676254966329717 kg/s
msc:  0.014503665572394816 kg/s
v:  2620.0342431715085 cm/s
v:  2620.034243171508 cm/s


In [5]:
# calculates velocity
# 3mm-bypass gap (distance between channels)
rlc = 1.588/2  # cm
rsc = 1.27/2  # cm
rho = 4.36992  # [kg/m3] 70 bar, 490 C
rho /= 1e6  # [kg/cm3]
bg = 0.15
f = 18 * np.tan(np.pi/6)

m = 0.2  # kg/s
Ag = f * bg + 0.125 * bg * bg * np.tan(np.pi/6) # Area of half of the gap width between channels

mlc = m/(8.5*rlc**2 + 0.5*rsc**2 + Ag/np.pi)*rlc**2
print('mlc: ', mlc*1e3, 'kg/s')
msc = m/(8.5*rlc**2 + 0.5*rsc**2 + Ag/np.pi)*rsc**2
print('msc: ', msc*1e3, 'kg/s')
mg = m/(8.5*rlc**2 + 0.5*rsc**2 + Ag/np.pi)*Ag/np.pi
print('mg: ', mg*1e3, 'kg/s')

print('1/2 mlc: ', 0.5*mlc*1e3, 'kg/s')
print('1/2 msc: ', 0.5*msc*1e3, 'kg/s')
print('1/2 mg: ', 0.5*mg*1e3, 'kg/s')

v = mlc/rho/np.pi/rlc**2
print('v: ', v, 'cm/s')
v = msc/rho/np.pi/rsc**2
print('v: ', v, 'cm/s')
v = mg/rho/Ag
print('v: ', v, 'cm/s')

# m2 = [m[0]/2, m[1]/2, m[2]/2, m[3]/2, m[4]/2, m[5]/2, m[6], m[7], m[8], m[9]/2, m[10], m[11], m[12]/2]

mlc:  20.816666145173254 kg/s
msc:  13.314278065319053 kg/s
mg:  16.401198733367853 kg/s
1/2 mlc:  10.408333072586627 kg/s
1/2 msc:  6.657139032659527 kg/s
1/2 mg:  8.200599366683926 kg/s
v:  2405.175731619084 cm/s
v:  2405.1757316190838 cm/s
v:  2405.175731619084 cm/s


## Incompressible flow:

The Re is not affected by the temperature.

$\frac{dp}{dz} + \frac{1}{2} \rho v^2 f \frac{P_f}{A} = 0$ (3.81 [1])

$\Delta p = \frac{1}{2} \rho v^2 f \frac{P_f L}{A}$ (3.82 [1])

Reordering:

$\Delta p = \frac{1}{\rho} \left( \frac{\dot{m}}{A} \right)^2 f \frac{2 L}{D_h}$ (3.82 [1])

In [2]:
# calculates the mass flow in each channel
# no gap
L = 793  # cm
Dhi = 1.588 * np.ones(13)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5])
Ai = np.pi * (Dhi/2)**2
# print(Ai)
rho = 4.3679e-6  # kg/cm3
mu = 3.8236e-5  # Pa.s = kg/m/s
mu /= 1e2  # kg/cm/s
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # print('Re: ', Re)
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    # print('A: ', A)
    B = (37530/Re)**16
    # print('B: ', B)
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    # fi = 0.184/Re**0.2
    # print('fi: ', fi)
    # dp [1e-2 Pa]
    Bi = 1/rho/Ai**2*fi*2/Dhi*L
    # print('Bi: ', Bi)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    dp = sqrtdp**2
    print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print(k)
print('mi: ', mi)
print('mt: ', sum(Ci*mi))

dp:  703.3475691194249
dp:  703.3323166637149
dp:  703.3351740747352
dp:  703.3354503368706
dp:  703.3354745058261
5
mi:  [0.0125151  0.02279323 0.02279323 0.02279323 0.02279323 0.02279323
 0.02279323 0.02279323 0.02279323 0.02279323 0.02279323 0.02279323
 0.02279323]
mt:  0.2


In [18]:
# calculates the mass flow in each channel
# With 3mm gap (distance between channels)
L = 793  # cm
Dhi = 1.588 * np.ones(14)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5, 0.5])
Ai = np.pi * (Dhi/2)**2

bg = 0.3  # gap [cm]
f = 18 * np.tan(np.pi/6)  # [cm]
Ai[-1] = f * bg + 0.25 * bg * bg * np.tan(np.pi/6) # Area of the total width between channels
Ph = 2 * f + bg/np.cos(np.pi/6)
Dhi[-1] = 4 * Ai[-1]/Ph

rho = 4.3679e-6  # kg/cm3

# mu = 3.8236e-5  # Pa.s = kg/m/s
Tave = (490 + 950) / 2
file = pd.read_csv('heproperties')
temp = file['Temperature (K)'].tolist()
m_c = np.array(file['Viscosity (Pa*s)'].tolist())
mu = np.interp(Tave+273, temp, m_c)

print(mu)

mu /= 1e2  # kg/cm/s
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # print('Re: ', Re)
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    # print('A: ', A)
    B = (37530/Re)**16
    # print('B: ', B)
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    # fi = 0.184/Re**0.2
    # fi = 0.001375*(1+(20e3*eps/Dhi+1e6/Re)**(1/3))
    # print('fi: ', fi)
    # dp [1e-2 Pa]
    Bi = 1/rho/Ai**2*fi*2/Dhi*L
    # print('Bi: ', Bi)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    dp = sqrtdp**2
    print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print(k)
print('mi: ', mi)
print('mt: ', sum(Ci*mi))

v = mi/rho/Ai
print('v: ', np.round(v,0), 'cm/s')

m = mi
m2 = [m[0]/2, m[1]/2, m[2]/2, m[3]/2, m[4]/2, m[5]/2, m[6], m[7], m[8], m[9]/2, m[10], m[11], m[12]/2, m[13]/2]
m2 = np.array(m2)*1000
print('Real mass flow for each channel: ', np.round(m2, 2))

4.600224e-05
dp:  670.2325554805238
dp:  669.6171586891293
dp:  669.7277043766117
dp:  669.7426146064846
dp:  669.7444216189936
dp:  669.7446383652273
dp:  669.7446643505231
7
mi:  [0.01196085 0.02181975 0.02181975 0.02181975 0.02181975 0.02181975
 0.02181975 0.02181975 0.02181975 0.02181975 0.02181975 0.02181975
 0.02181975 0.01710342]
mt:  0.19999999999999996
v:  [2162. 2522. 2522. 2522. 2522. 2522. 2522. 2522. 2522. 2522. 2522. 2522.
 2522. 1251.] cm/s
Real mass flow for each channel:  [ 5.98 10.91 10.91 10.91 10.91 10.91 21.82 21.82 21.82 10.91 21.82 21.82
 10.91  8.55]


## Incompressible flow with temperature effects

In [19]:
# calculates the mass flow in each channel
# With 3mm gap (distance between channels)
L = 793  # cm
Dhi = 1.588 * np.ones(14)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5, 0.5])
Ai = np.pi * (Dhi/2)**2

bg = 0.3  # gap [cm]
f = 18 * np.tan(np.pi/6)  # [cm]
Ai[-1] = f * bg + 0.25 * bg * bg * np.tan(np.pi/6) # Area of the total width between channels
Ph = 2 * f + bg/np.cos(np.pi/6)
Dhi[-1] = 4 * Ai[-1]/Ph

rho = 4.3679e-6  # kg/cm3
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s

Ti = 490
# Initial To guess:
# file = pd.read_csv('incom.csv')
# Next iterations:
file = pd.read_csv('incom-temp-2.csv')
To = np.zeros(14)
for i in range(len(To)):
    To[i] = file['outcool'+str(i+1)].tolist()[0]
    
Tave = (Ti+To)/2
# print(Tave)
file = pd.read_csv('heproperties')
temp = file['Temperature (K)'].tolist()
m_c = np.array(file['Viscosity (Pa*s)'].tolist())

mu = np.interp(Tave+273, temp, m_c)
# print(mu)
mu /= 1e2  # kg/cm/s

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # print('Re: ', Re)
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    # print('A: ', A)
    B = (37530/Re)**16
    # print('B: ', B)
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    # fi = 0.184/Re**0.2
    # fi = 0.001375*(1+(20e3*eps/Dhi+1e6/Re)**(1/3))
    # print('fi: ', fi)
    # dp [1e-2 Pa]
    Bi = 1/rho/Ai**2*fi*2/Dhi*L
    # print('Bi: ', Bi)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    dp = sqrtdp**2
    # print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print('number of iterations: ', k)
# print('mi: ', mi)
print('mt: ', sum(Ci*mi))

v = mi/rho/Ai
print('v: ', np.round(v, 0), 'cm/s')

m = mi
m2 = [m[0]/2, m[1]/2, m[2]/2, m[3]/2, m[4]/2, m[5]/2, m[6], m[7], m[8], m[9]/2, m[10], m[11], m[12]/2, m[13]/2]
m2 = np.array(m2)*1000
print('Real mass flow for each channel: ', np.round(m2, 2))

number of iterations:  7
mt:  0.20000000000000004
v:  [2157. 2520. 2521. 2522. 2525. 2519. 2520. 2521. 2523. 2521. 2523. 2526.
 2525. 1251.] cm/s
Real mass flow for each channel:  [ 5.97 10.9  10.9  10.91 10.92 10.9  21.8  21.81 21.83 10.91 21.82 21.85
 10.92  8.55]


## Low-Mach number flow

$\Delta P = \frac{\dot{m}^2}{2 \rho A^2} \left[ \frac{4 f L (T_i+T_o)}{2 D T_i} + \frac{T_o-T_i}{T_i} \right]$


In [4]:
# calculates the mass flow in each channel
# No gap - doesn't consider the viscosity dependent of the temperature
L = 793  # cm
Dhi = 1.588 * np.ones(13)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5])
Ai = np.pi * (Dhi/2)**2
# print(Ai)
rho = 4.3679e-6  # kg/cm3
mu = 3.8236e-5  # Pa.s = kg/m/s
mu /= 1e2  # kg/cm/s
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s
Ti = 490  # C
# To = 950 * np.ones(13)
To = np.array([983.5, 956.3, 948.6, 945.3, 934.6, 958.7, 950.4, 946.3, 945.3, 946, 941.1, 927.1, 931.6])

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    B = (37530/Re)**16
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    # fi = 0.184/Re**0.2

    Bi = 1/2/rho/Ai**2*(2*fi*L*(Ti+To)/Dhi/Ti + (To-Ti)/Ti)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    # dp [1e-2 Pa]
    dp = sqrtdp**2
    print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print(k)
print('mi: ', mi)
print('mt: ', sum(Ci*mi))

dp:  1043.7164916242752
dp:  1043.6902965283934
dp:  1043.6952758637426
dp:  1043.6957529604836
dp:  1043.6957944090034
5
mi:  [0.0123536  0.02269039 0.02275818 0.02278741 0.02288291 0.02266937
 0.02274228 0.02277854 0.02278741 0.0227812  0.02282476 0.02295053
 0.02290989]
mt:  0.2


In [10]:
# calculates the mass flow in each channel
# With gap - doesn't consider the viscosity dependent of the temperature
# gap area and Ph wrong
L = 793  # cm
Dhi = 1.588 * np.ones(14)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5, 0.5])
Ai = np.pi * (Dhi/2)**2

bg = 0.3  # gap [cm]
f = 18 * np.tan(np.pi/6)  # [cm]
Ai[-1] = f * bg + 0.5 * bg * bg * np.tan(np.pi/6)
Ph = 2 * f + bg * np.tan(np.pi/6) + bg + bg/np.cos(np.pi/6)
Dhi[-1] = 4 * Ai[-1]/Ph

rho = 4.3679e-6  # kg/cm3
mu = 3.8236e-5  # Pa.s = kg/m/s
mu /= 1e2  # kg/cm/s
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s

Ti = 490  # C
To = np.array([1002.66, 972.18, 953.4, 925.5, 873.1, 976.22, 960.82, 939.4, 905.48, 942.93, 913.04, 861.17, 871.28, 890.95])

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # print('Re: ', Re)
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    # print('A: ', A)
    B = (37530/Re)**16
    # print('B: ', B)
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    Bi = 1/2/rho/Ai**2*(2*fi*L*(Ti+To)/Dhi/Ti + (To-Ti)/Ti)
    # print('Bi: ', Bi)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    dp = sqrtdp**2
    print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print(k)
print('mi: ', mi)
print('mt: ', sum(Ci*mi))

dp:  992.2949447898286
dp:  991.4337848312729
dp:  991.6070900742515
dp:  991.6302496899804
dp:  991.6330465664844
dp:  991.6333812199407
dp:  991.633421261889
7
mi:  [0.01170162 0.02155405 0.02165816 0.02179659 0.02215299 0.02152796
 0.02162353 0.02172609 0.02190817 0.02170766 0.02186476 0.0222242
 0.02213348 0.01707398]
mt:  0.2


In [25]:
# calculates the mass flow in each channel
# With gap - viscosity(T)
# With 3mm gap (distance between channels)
L = 793  # cm
Dhi = 1.588 * np.ones(14)
Dhi[0] = 1.270
Ci = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1, 0.5, 0.5])
Ai = np.pi * (Dhi/2)**2

bg = 0.3  # gap [cm]
f = 18 * np.tan(np.pi/6)  # [cm]
Ai[-1] = f * bg + 0.25 * bg * bg * np.tan(np.pi/6) # Area of the total width between channels
Ph = 2 * f + bg/np.cos(np.pi/6)
Dhi[-1] = 4 * Ai[-1]/Ph

rho = 4.3679e-6  # kg/cm3
mu = 3.8236e-5  # Pa.s = kg/m/s
mu /= 1e2  # kg/cm/s
eps = 10e-6  # m
eps *= 1e2  # cm
mT = 0.2  # kg/s

Ti = 490
# Fixed To
# To = np.array([1002.66, 972.18, 953.4, 925.5, 873.1, 976.22, 960.82, 939.4, 905.48, 942.93, 913.04, 861.17, 871.28, 890.95])
# Initial To guess:
# file = pd.read_csv('incom.csv')
# Next iterations:
file = pd.read_csv('accel-3.csv')
To = np.zeros(14)
for i in range(len(To)):
    To[i] = file['outcool'+str(i+1)].tolist()[0]

Tave = (Ti+To)/2
file = pd.read_csv('heproperties')
temp = file['Temperature (K)'].tolist()
m_c = np.array(file['Viscosity (Pa*s)'].tolist())
mu = np.interp(Tave+273, temp, m_c)
mu /= 1e2  # kg/cm/s

mi = mT * Ai/sum(Ai*Ci)  # initial guess
# print(mi)
dp_ant = 0
converge = False
k = 0
while converge == False and k < 20:
    Re = mi/Ai * Dhi/mu  # mi[kg/s], Ai[cm2], Dh[cm], mu[kg/cm/s]
    # print('Re: ', Re)
    # Eq 3.9 [2]: Churchill
    A = (-2.457 * np.log((7/Re)**0.9+0.27*eps/Dhi))**16
    # print('A: ', A)
    B = (37530/Re)**16
    # print('B: ', B)
    fi = 8*( (8/Re)**12 + 1/(A+B)**1.5 )**(1/12)
    Bi = 1/2/rho/Ai**2*(2*fi*L*(Ti+To)/Dhi/Ti + (To-Ti)/Ti)
    # print('Bi: ', Bi)
    sqrtdp = mT/(sum(Ci/np.sqrt(Bi)))
    dp = sqrtdp**2
    print('dp: ', dp)
    if np.abs(dp-dp_ant) < 1e-4:
        converge = True
    mi = sqrtdp/np.sqrt(Bi)
    k += 1
    dp_ant = dp

print(k)
# print('mi: ', mi)
print('mt: ', sum(Ci*mi))

v = mi/rho/Ai
print('v: ', np.round(v, 0), 'cm/s')

m = mi
m2 = [m[0]/2, m[1]/2, m[2]/2, m[3]/2, m[4]/2, m[5]/2, m[6], m[7], m[8], m[9]/2, m[10], m[11], m[12]/2, m[13]/2]
m2 = np.array(m2)*1000
print('Real mass flow for each channel: ', np.round(m2, 2))

dp:  991.7257865010434
dp:  990.897635739919
dp:  991.0646152987715
dp:  991.0867552859971
dp:  991.0894081779962
dp:  991.0897231784209
dp:  991.089760584935
7
mt:  0.2
v:  [2107. 2484. 2499. 2519. 2564. 2481. 2494. 2509. 2534. 2507. 2528. 2574.
 2563. 1262.] cm/s
Real mass flow for each channel:  [ 5.83 10.75 10.81 10.9  11.09 10.73 21.58 21.71 21.92 10.84 21.87 22.26
 11.08  8.63]


## References:

* [1] Thermal and flow design of Helium-cooled reactors. 1984.
* [2] Huning. A STEADY STATE THERMAL HYDRAULIC ANALYSIS METHOD FOR PRISMATIC GAS REACTORS. 2014.
* [] Tak et al. A practical method for whole-core thermal analysis of a prismatic gas-cooled reactor. 2012.

In [3]:
# Results

ref=np.array([5.88, 10.80, 10.85, 10.91, 11.08, 10.80, 21.58, 21.67, 21.83, 10.88, 21.81, 22.20, 11.10, 8.28])
cs1=np.array([6.66, 10.41, 10.41, 10.41, 10.41, 10.41, 20.82, 20.82, 20.82, 10.41, 20.82, 20.82, 10.41, 8.20])
cs2=np.array([5.98, 10.91, 10.91, 10.91, 10.91, 10.91, 21.82, 21.82, 21.82, 10.91, 21.82, 21.82, 10.91, 8.55])
cs3=np.array([5.97, 10.90, 10.90, 10.91, 10.92, 10.90, 21.80, 21.81, 21.83, 10.91, 21.82, 21.85, 10.92, 8.55])
cs4=np.array([5.83, 10.75, 10.81, 10.90, 11.09, 10.73, 21.58, 21.71, 21.92, 10.84, 21.87, 22.26, 11.08, 8.63])